<a href="https://colab.research.google.com/github/umsi-data-science/si370/blob/master/SI%20370%3A%20Day%2003%3A%20Loading%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SI370 Day 3: Loading data in pandas



## Learning Objectives
* load CSV files
* load JSON files
* use pd.read_html to extract tables from web pages
* load data from simple APIs 
* load data from a SQL database


IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [0]:
MY_UNIQNAME = '?'

## Exercise 1

a. Sign up for a Kaggle account (https://www.kaggle.com/).  Record your Kaggle username in the following markdown cell


Replace this with your Kaggle username

b. Browse the Kaggle datasets (https://www.kaggle.com/datasets) and list two or three that you find interesting.  Explain why you find them interesting.

Insert your answer here.

# Today's focus: Loading (and manipulating) data using pandas

In [0]:
import pandas as pd

Recall the ```pd.read_csv``` function that we used to load data sets in previous classes:

In [0]:
menu = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/si370/master/data/menu.csv') 

That works great for well-formatted CSV files, but what happens when you get something that looks like the ```data/avocado_eu.csv``` file.
Go ahead and browse that in JupyterLab's CSV browser.

You'll notice a new drop-down menu labelled "Delimiter".  Go ahead and change that to ```;```.

Referring back to your readings and the [read_csv documentation online](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html), complete the following exercise


Read the data/avocado_eu.csv file into a pandas DataFrame and show the first 5 rows.


In [0]:
avocado = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/si370/master/data/avocado_eu.csv', delimiter=';')
avocado.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,"1,33","64236,62","1036,74","54454,85","48,16","8696,87","8603,62","93,25","0,0",conventional,2015,Albany
1,1,2015-12-20,"1,35","54876,98","674,28","44638,81","58,33","9505,56","9408,07","97,49","0,0",conventional,2015,Albany
2,2,2015-12-13,"0,93","118220,22","794,7","109149,67","130,5","8145,35","8042,21","103,14","0,0",conventional,2015,Albany
3,3,2015-12-06,"1,08","78992,15","1132,0","71976,41","72,58","5811,16","5677,4","133,76","0,0",conventional,2015,Albany
4,4,2015-11-29,"1,28","51039,6","941,48","43838,39","75,78","6183,95","5986,26","197,69","0,0",conventional,2015,Albany


You'll notice that, unless you did something special in the previous read_csv invocation, the decimal points don't look quite right.  Go ahead and find the right option to convert commas to periods when loading a CSV file.

## Exercise 2 (1 point):
Read the data/avocado_eu.csv file using the correct delimiter and decimal character into a dataframe and show the first 5 rows:

In [0]:
avocado = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/si370/master/data/avocado_eu.csv', delimiter=';', decimal=',')
avocado.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


# Loading JSON data

In addition to CSV files, JSON (JavaScript Object Notation) files or data is commonly used.  

In [0]:
nfl_football_players = pd.read_json('https://raw.githubusercontent.com/umsi-data-science/si370/master/data/nfl_football_profiles.json')

In [0]:
nfl_football_players.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN
3,18182,Steve Ramsey,QB,6-2,210.0,None,1948-04-22,"Dallas, TX",1999-10-15,North Texas,"W.W. Samuell, TX",New Orleans Saints,5.0,126.0,1970.0,None,NaN
4,16250,Cory Nelms,CB,6-0,195.0,None,1988-02-27,"Neptune, NJ",None,Miami (FL),"Neptune, NJ",None,NaN,NaN,NaN,None,NaN


And, just for fun, show the player with the highest Current Salary from that dataset:

In [0]:
nfl_football_players.sort_values('current_salary', ascending=False).head(1)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
6454,721,Jeremiah Attaochu,OLB,6-3,252.0,Los Angeles Chargers,1993-01-17,"Ibadan, Nigeria",None,Georgia Tech,"Archbishop Carroll, DE",San Diego Chargers,2.0,50.0,2014.0,"993,150",NaN


## Fixing up the data
Assuming you did something like sort_values on one of the original columns, you probably got the wrong result.

Looking a bit more closely at the results, you'll notice that the current_salary column.  Remembering that we have made the shift from pythonic to pandorable, we can leverage the impressive-sounding "vectorized string functions" mentioned in Section XXX of the McKinney book.  Specifically, we can use the str.replace(...) method.  Note that had we use read_csv to load the file we could have used the ```thousands=``` option and avoided all this, but sometimes data doesn't come in a convenient format.

One way to apply functions is to operate on a column and then assign the results to another column.  For example, if we wanted to eliminate commas, we could replace them with null strings


In [0]:
nfl_football_players['current_salary'].str.replace(',', '')

0           None
1           None
2        1075000
3           None
4           None
          ...   
25038       None
25039       None
25040       None
25041       None
25042       None
Name: current_salary, Length: 25043, dtype: object

And assign the results to a column in the original dataframe (in this case I'm calling the column current_salary_nocommas)

In [0]:
nfl_football_players['current_salary_nocommas'] = nfl_football_players['current_salary'].str.replace(',', '')

But you'll notice that the type of the column is string, and we want to convert it to a float so we can sort it numerically.  So we can use the astype() function to convert it:

In [0]:
nfl_football_players['current_salary_cleaned'] = nfl_football_players['current_salary_nocommas'].astype(float)

In [0]:
nfl_football_players.head(2)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN,None,NaN


And now we can re-run our command to sort by salary and get the correct result:

In [0]:
nfl_football_players.sort_values('current_salary_cleaned', ascending=False).head(1)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
17756,4644,Kirk Cousins,QB,6-3,214.0,Washington Redskins,1988-08-19,"Holland, MI",None,Michigan St.,"Holland Christian, MI",Washington Redskins,4.0,102.0,2012.0,"23,943,600",NaN,23943600,23943600.0


## Dropping missing values

In addition to the "all" or "any" functionality described in McKinney section 7.1, it's sometimes useful to drop a row only if a certain column or columns have missing data.  To do this, use the subset= option with dropna().  So, for example, to drop all players for whom we do not have salary information, we could use the following code:

In [0]:
nfl_football_players_salaries = nfl_football_players.dropna(subset=['current_salary_cleaned'])

In [0]:
nfl_football_players_salaries.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN,1075000,1075000.0
6,2701,Preston Brown,ILB,6-1,251.0,Buffalo Bills,1992-10-27,"Cincinnati, OH",None,Louisville,"Northwest, OH",Buffalo Bills,3.0,73.0,2014.0,"1,762,000",NaN,1762000,1762000.0
13,3966,Frank Clark,DE,6-2,270.0,Seattle Seahawks,1993-06-14,"Cleveland, OH",None,Michigan,"Glenville, OH",Seattle Seahawks,2.0,63.0,2015.0,"774,294",NaN,774294,774294.0
37,19449,Emmanuel Sanders,WR,5-11,186.0,Denver Broncos,1987-03-16,"Bellville, TX",None,SMU,"Bellville, TX",Pittsburgh Steelers,3.0,82.0,2010.0,"6,750,000",NaN,6750000,6750000.0
53,17858,Bilal Powell,RB,5-11,204.0,New York Jets,1988-10-27,"Lakeland, FL",None,Louisville,"Lake Gibson, FL",New York Jets,4.0,126.0,2011.0,"3,750,000",NaN,3750000,3750000.0


# Scraping Tables from HTML

The ```pd.read_html``` function returns a list of DataFrames read from an HTML source.  The following line will return a list of DataFrames from https://en.wikipedia.org/wiki/List_of_largest_sports_contracts

In [0]:
contracts_scraped = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_sports_contracts',header=0)

In [0]:
len(contracts_scraped)

1

To get the first table, you'll need to pull off the 0th element:

In [0]:
contracts = contracts_scraped[0]
contracts.head()

,Rank,Player,Team/Streaming Service,Sport,Length of contract,Contract value (USD),Average per year (USD),Average per fixture (USD),Ref
0,1,Mike Trout,Los Angeles Angels,Baseball,12 years (2019–2030),"$430,000,000","$35,833,333","$221,193.42",[1]
1,2,Canelo Álvarez,DAZN*,Boxing,5 years (2018–2023),"$365,000,000","$73,000,000","$33,181,818.18",[2]
2,3,Bryce Harper,Philadelphia Phillies,Baseball,13 years (2019–2031),"$330,000,000","$25,384,615","$156,695.16",[3]
3,4,Giancarlo Stanton,Miami Marlins*,Baseball,13 years (2015–2027),"$325,000,000","$25,000,000","$154,320.99",[4]
4,5,Gerrit Cole,New York Yankees,Baseball,9 years (2020-2028),"$324,000,000","$36,000,000","$222,222.22",[5]


## Exercise 3 (2 points): 

Count the number of players from each sport in the List of Largest Sports Contracts 

Hint:  see value_counts() description above

In [0]:
contracts['Sport'].value_counts()

Baseball                50
Basketball              33
American football       11
American Football        2
Auto racing              1
Boxing                   1
Association football     1
Name: Sport, dtype: int64

For the final exercise, we're going to return to the nfl_football_players dataframe we created earlier.  The following question looks deceptively easy.
We'd like you to tackle this on in your groups.  We'd like you to first use the whiteboard to come up with a plan to tackle this exercise. 
You should first list the steps you'll follow and the highlight the steps that you think you'll need to look up extra documentation for.  Members of the first group to complete this exercise **and help another group complete it by explaining how to do it** will receive **two bonus marks**.

## Exercise 4 (5 points): 
Create a new dataframe that contains all the columns in the nfl_football_players dataframe as well as an additional column that contains each player's height in centimeters. Show the first 5 rows of your result.

hint: 1 inch = 2.54 cm

hint: you can use the vectorized string function str.split() to separate feet and inches from the original dataframe column

hint: remember to cast strings to numeric types if you're going to perform math on them

hint: you might want to create an intermediate (temporary) DataFrame to help you keep things clear instead of attempting to do 
this in one line 

In [0]:
nfl_football_players.columns

Index(['player_id', 'name', 'position', 'height', 'weight', 'current_team',
       'birth_date', 'birth_place', 'death_date', 'college', 'high_school',
       'draft_team', 'draft_round', 'draft_position', 'draft_year',
       'current_salary', 'hof_induction_year', 'current_salary_nocommas',
       'current_salary_cleaned'],
      dtype='object')

In [0]:
nfl_football_players.height.head(5)

0    6-0
1    6-3
2    6-3
3    6-2
4    6-0
Name: height, dtype: object

In [0]:
heights = nfl_football_players.height.str.split('-')

In [0]:
heights.head()

0    [6, 0]
1    [6, 3]
2    [6, 3]
3    [6, 2]
4    [6, 0]
Name: height, dtype: object

In [0]:
heights = nfl_football_players.height.str.split('-', expand=True)

In [0]:
heights.head()

,0,1
0,6,0
1,6,3
2,6,3
3,6,2
4,6,0


In [0]:
heights = nfl_football_players.height.str.split('-', expand = True)
nfl_football_players['cm'] = (heights[0].astype(float)*12+heights[1].astype(float))*2.54

In [0]:
nfl_football_players.head(1)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned,cm
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN,None,NaN,182.88


# APIs and requests
You've covered the ```requests``` package in previous courses.  This example shows what you can do with an API that returns JSON:

In [0]:
import requests

In [0]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [0]:
resp = requests.get(url)
resp

<Response [200]>

In [0]:
data = resp.json()

In [0]:
data[0]['title']

'REF: simplify concat_datetime'

In [0]:
issues = pd.DataFrame(data)
issues.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,user,labels,state,locked,assignee,assignees,milestone,comments,created_at,updated_at,closed_at,author_association,pull_request,body
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/33526,598920881,MDExOlB1bGxSZXF1ZXN0NDAyNjY5NDQ2,33526,REF: simplify concat_datetime,"{'login': 'jbrockmendel', 'id': 8078968, 'node...","[{'id': 127681, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open,False,None,[],None,0,2020-04-13T14:27:46Z,2020-04-13T14:56:31Z,None,MEMBER,{'url': 'https://api.github.com/repos/pandas-d...,"By wrapping dt64/td64 ndarrays in DTA/TDA, we ..."
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/33525,598916702,MDExOlB1bGxSZXF1ZXN0NDAyNjY2MTI2,33525,TST: Add Categorical Series test,"{'login': 'dsaxton', 'id': 2658661, 'node_id':...",[],open,False,None,[],None,0,2020-04-13T14:20:13Z,2020-04-13T14:59:02Z,None,CONTRIBUTOR,{'url': 'https://api.github.com/repos/pandas-d...,Follow up to https://github.com/pandas-dev/pan...
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/33524,598897368,MDU6SXNzdWU1OTg4OTczNjg=,33524,ENH: Canonic SI frequency,"{'login': 'JoElfner', 'id': 38665102, 'node_id...","[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open,False,None,[],None,0,2020-04-13T13:43:44Z,2020-04-13T13:43:44Z,None,CONTRIBUTOR,NaN,"Since pandas is widely used for data science, ..."
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/33522,598840503,MDExOlB1bGxSZXF1ZXN0NDAyNjA1ODY0,33522,BUG: Fixed concat with reindex and extension t...,"{'login': 'TomAugspurger', 'id': 1312546, 'nod...","[{'id': 849023693, 'node_id': 'MDU6TGFiZWw4NDk...",open,False,None,[],{'url': 'https://api.github.com/repos/pandas-d...,2,2020-04-13T11:35:47Z,2020-04-13T15:11:32Z,None,CONTRIBUTOR,{'url': 'https://api.github.com/repos/pandas-d...,Closes https://github.com/pandas-dev/pandas/is...
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/33521,598832866,MDExOlB1bGxSZXF1ZXN0NDAyNTk5ODUy,33521,TYP: disallow decorator preserves function sig...,"{'login': 'simonjayhawkins', 'id': 13159005, '...","[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open,False,None,[],None,0,2020-04-13T11:19:21Z,2020-04-13T15:12:08Z,None,MEMBER,{'url': 'https://api.github.com/repos/pandas-d...,xref #33455


In [0]:
issues.columns

Index(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url',
       'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels',
       'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments',
       'created_at', 'updated_at', 'closed_at', 'author_association',
       'pull_request', 'body'],
      dtype='object')

In [0]:
issues = pd.DataFrame(data, columns=['number', 'title','labels', 'state'])
issues.head()

,number,title,labels,state
0,33526,REF: simplify concat_datetime,"[{'id': 127681, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
1,33525,TST: Add Categorical Series test,[],open
2,33524,ENH: Canonic SI frequency,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,33522,BUG: Fixed concat with reindex and extension t...,"[{'id': 849023693, 'node_id': 'MDU6TGFiZWw4NDk...",open
4,33521,TYP: disallow decorator preserves function sig...,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open


# Accessing databases
This section is preparation for later in the course, but it's useful to take a look at it in light of the topic of "loading data".

We'll use this opportunity to gauge where we're at in terms of understanding SQL.

---



In [0]:
import sqlalchemy as sqla

In [0]:
!pip install pymysql

In [0]:
#@title Database parameters { form-width: "25%" }
endpoint = "database-1.cgeyvh9t1svz.us-east-1.rds.amazonaws.com" #@param {type:"string"}
username = "si370wn2020" #@param {type:"string"}
password = "secret" #@param {type:"string"}
database = "goodreads" #@param {type:"string"}

db = sqla.create_engine(f'mysql+pymysql://{username}:{password}@{endpoint}/{database}')

In [0]:
df = pd.read_sql('select count(*) from ratings',db)
df.head()

,count(*)
0,5976479


In [0]:
df = pd.read_sql('select rating, count(*) from ratings group by rating', db)

In [0]:
df.head()

,rating,count(*)
0,1,124195
1,2,359257
2,3,1370916
3,4,2139018
4,5,1983093
